In [20]:
import matplotlib.pyplot as plt
from math import exp
from scipy.stats import norm
from scipy import stats
import seaborn as sns
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import warnings
from stats

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 300)
plt.style.use('seaborn')
sns.set(style="white")

In [6]:
df = pd.read_csv('day.csv')

In [7]:
df.drop(labels='instant',axis=1,inplace=True)

In [8]:
seasons_num = df['season'].unique()

In [9]:
seasons = ['spring', 'summer', 'fall', 'winter']

In [10]:
seasons_dict = dict(zip(seasons_num, seasons))

In [11]:
df['seasons'] = df['season'].map(seasons_dict)

In [12]:
df = pd.concat([df, pd.get_dummies(df['seasons'])], 1)

In [13]:
weather_num = df['weathersit'].unique()
weathersits = ['clear', 'misty', 'light_storm', 'heavy_storm']
weather_dict = dict(zip(weather_num, weathersits))
df['weather'] = df['weathersit'].map(weather_dict)
df = pd.concat([df, pd.get_dummies(df['weather'])], 1)

In [14]:
def columnize(df, column, new_vals, new_col):
    list1 = df[column].unique()
    dict1 = dict(zip(list1,new_vals))
    df[new_col] = df[column].map(dict1)
    df = pd.concat([df, pd.get_dummies(df[new_col])], 1)
    return df

In [15]:
df.dtypes

dteday          object
season           int64
yr               int64
mnth             int64
holiday          int64
weekday          int64
workingday       int64
weathersit       int64
temp           float64
atemp          float64
hum            float64
windspeed      float64
casual           int64
registered       int64
cnt              int64
seasons         object
fall             uint8
spring           uint8
summer           uint8
winter           uint8
weather         object
clear            uint8
light_storm      uint8
misty            uint8
dtype: object

In [16]:
df['weekday'].unique()

array([6, 0, 1, 2, 3, 4, 5])

In [17]:
df = columnize(df, 'weekday', ['sat','sun','mon','tues','wed','thurs','fri'], 'weekdays')

In [18]:
df.columns

Index(['dteday', 'season', 'yr', 'mnth', 'holiday', 'weekday', 'workingday',
       'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'casual',
       'registered', 'cnt', 'seasons', 'fall', 'spring', 'summer', 'winter',
       'weather', 'clear', 'light_storm', 'misty', 'weekdays', 'fri', 'mon',
       'sat', 'sun', 'thurs', 'tues', 'wed'],
      dtype='object')

In [19]:
lr_model = ols(formula='cnt~hum+winter', data=df).fit()

lr_model.summary()

NameError: name 'ols' is not defined

In [ ]:
def CorrMtx(df, dropDuplicates = True):

    # Your dataset is already a correlation matrix.
    # If you have a dateset where you need to include the calculation
    # of a correlation matrix, just uncomment the line below:
    # df = df.corr()

    # Exclude duplicate correlations by masking uper right values
    if dropDuplicates:    
        mask = np.zeros_like(df, dtype=np.bool)
        mask[np.triu_indices_from(mask)] = True

    # Set background color / chart style
    sns.set_style(style = 'white')

    # Set up  matplotlib figure
    f, ax = plt.subplots(figsize=(11, 9))

    # Add diverging colormap from red to blue
    cmap = sns.diverging_palette(250, 10, as_cmap=True)

    # Draw correlation plot with or without duplicates
    if dropDuplicates:
        sns.heatmap(df, mask=mask, cmap=cmap, 
                square=True,
                linewidth=.5, cbar_kws={"shrink": .5}, ax=ax)
    else:
        sns.heatmap(df, cmap=cmap, 
                square=True,
                linewidth=.5, cbar_kws={"shrink": .5}, ax=ax)


CorrMtx(df.corr(), dropDuplicates = True)

In [ ]:
plt.scatter(df.temp, df.cnt)

In [ ]:
df['cnt'].describe()

In [ ]:
sns.pairplot(df, diag_kind="kde", height=2.5)

In [ ]:
feats = list(df.select_dtypes(include=['int64','float64']).columns)

In [ ]:
n = 4
row_groups= [feats[i:i+n] for i in range(0, len(feats), n)]

In [ ]:
for i in row_groups:
    pp = sns.pairplot(data=df, y_vars=['cnt'],x_vars=i, kind="reg", height=3)

In [ ]:
def day_season(df,season,day):
    df[f'{season}_{day}_cnt'] = df[f'{season}']*df['cnt']*df[f'{day}']
    return

In [ ]:
for s in ['summer','fall','winter','spring']:
    for d in ['sat','sun','mon','tues','wed','thurs','fri']:
        day_season(df,s,d)

In [ ]:
def s_day_df(df,season,day):
    return df[df[f'{season}_{day}_cnt']>0]

In [ ]:
import itertools

In [ ]:
seasons = ['summer','fall','winter','spring']
days = ['sat','sun','mon','tues','wed','thurs','fri']

In [ ]:
sd = {}
for s in seasons:
    for d in days:
        sd[f'{s}_{d}'] = s_day_df(df,s,d)

In [ ]:
def day_graphs(df,season,day):
    feats = list(df.select_dtypes(include=['int64','float64']).columns)
    n = 4
    row_groups= [feats[i:i+n] for i in range(0, len(feats), n)]
    for i in row_groups:
        pp = sns.pairplot(data=df, y_vars=[f'{season}_{day}_cnt'],x_vars=i, kind="reg", height=3)
    return

In [ ]:
day_graphs(sd['spring_sun'],'spring','sun')

In [ ]:
df['spring_cnt'] = df['spring']*df['cnt']
df['winter_cnt'] = df['winter']*df['cnt']
df['summer_cnt'] = df['summer']*df['cnt']
df['fall_cnt'] = df['fall']*df['cnt']

In [ ]:
def season_df(df,season):
    return df[df[f'{season}_cnt']>0]

In [ ]:
wint = season_df(df,'winter')
summer = season_df(df,'summer')
fall = season_df(df,'fall')

In [ ]:
spr = df[df['spring_cnt']>0]
sns.pairplot(data=spr, y_vars=['spring_cnt'],x_vars=['temp'], kind="reg",height=4.5)

In [ ]:
model = ols(formula='spring_cnt~temp+hum+windspeed+clear+misty+light_storm+sun+mon+tues+wed+thurs+fri+sat', data=spr).fit()

model.summary()

In [ ]:
spr_feats = list(spr.select_dtypes(include=['int64','float64']).columns)

In [ ]:
def season_graphs(df,season):
    feats = list(df.select_dtypes(include=['int64','float64']).columns)
    n = 4
    row_groups= [feats[i:i+n] for i in range(0, len(feats), n)]
    for i in row_groups:
        pp = sns.pairplot(data=df, y_vars=[f'{season}_cnt'],x_vars=i, kind="reg", height=3)
    return

In [ ]:
season_graphs(spr,'spring')

In [ ]:
season_graphs(wint,'winter')

In [ ]:
season_graphs(summer,'summer')

In [ ]:
season_graphs(fall,'fall')

In [ ]:
model = ols(formula='cnt~dteday', data=df).fit()

model.summary()